# ImXPAD S540 detector at D2AM

This tutorial corresponds to the calibration the goniometer an ImXPAD detector
composed of 8 stripes of 7 modules, many of which are defective, on a goniometer.

This detector is mounted on the goniometer 2theta arm at the D2AM beam-line, French CRG
at the ESRF synchrotron. 

The raw data files are available at:
http://www.silx.org/pub/pyFAI/gonio/D2AM-15/

In [ ]:
%pylab nbagg

In [ ]:
import glob, os, time
start_time = time.time()
import fabio, pyFAI
from pyFAI.goniometer import GeometryTransformation, GoniometerRefinement, Goniometer
from pyFAI.gui import jupyter

In [ ]:
# Configure data location
DATA_DIRECTORY = "/workspace/valls/data/goniometer/D2AM-15"

In [ ]:
#Definition of the detector and deplay of an image and its mask:

d5 = pyFAI.detector_factory(DATA_DIRECTORY + "/D5Geom.h5")
print(d5.shape)

images = glob.glob(DATA_DIRECTORY + "/*.edf.gz")
fimg = fabio.open(images[-1])

for k,v in fimg.header.items():
    print(k, ": ", v)
    
f,ax=subplots(1,2)
ax[0].imshow(d5.mask, origin="lower")
ax[0].set_title("D5 Mask")
ax[1].imshow(numpy.arcsinh(fimg.data), cmap="inferno", origin="lower")
ax[1].set_title(fimg.filename)


In [ ]:
# Define wavelength and create our "large" LaB6 calibrant

wavelength = 0.495938 * 1e-10
from pyFAI.calibrant import Cell, Calibrant
c = Cell.cubic(4.1568260) 
c.save("LaB6", dmin=0.2)
LaB6 = Calibrant("LaB6.D")
LaB6.wavelength = wavelength
print("2theta max: ", numpy.degrees(LaB6.get_2th()[-1]))
print("Number of reflections: ", len(LaB6.get_2th()))

In [ ]:
# Use a few manually calibrated images:

npt_files = [ i for i in glob.glob(DATA_DIRECTORY + "/*.npt") if "new" not in i]
npt_files.sort()
npt_files[0]
print("Number of hand-calibrated images :", len(npt_files))

In [ ]:
# Definition of the goniometer translation function:
# The detector rotates vertically, around the horizontal axis, i.e. rot2. 
# Rotation both around axis 1 and axis 2 are allowed

goniotrans = GeometryTransformation(param_names = ["dist", "poni1", "poni2", 
                                                   "rot1", "rot2", "rot3", "scale1", "scale2" ],
                                    dist_expr="dist", 
                                    poni1_expr="poni1",
                                    poni2_expr="poni2", 
                                    rot1_expr="scale1 * pos +rot1", 
                                    rot2_expr="scale2 * pos + rot2", 
                                    rot3_expr="rot3")


#Definition of the function reading the goniometer angle from the filename of the image.

def get_angle(metadata):
    """Takes the angle from the first motor position and returns the angle of the goniometer arm"""
    return float(metadata["motor_pos"].split()[0])

print('filename', fimg.filename, "angle:",get_angle(fimg.header))

In [ ]:
# Definition of the geometry refinement: the parameter order is the same as the param_names

rot3 = numpy.pi/2
scale1 = -numpy.pi/180
scale2 = 0
param = {"dist":0.5, 
         "poni1":0.05, 
         "poni2":0.05, 
         "rot1":0,
         "rot2":0,
         "rot3": rot3,
         "scale1": scale1,
         "scale2": scale2,
        }
# Defines the bounds for some variables
bounds = {"dist": (0.2, 0.8), 
          "poni1": (0, 0.1),
          "poni2": (0, 0.1),
          "rot1": (-0.1, 0.1),
          "rot2": (-0.1, 0.1),
          "rot3": (rot3, rot3), #strict bounds on rot3
          #"scale1": (scale1, scale1),
          #"scale2": (scale2, scale2),
         }
gonioref = GoniometerRefinement(param, #initial guess
                                bounds=bounds,
                                pos_function=get_angle,
                                trans_function=goniotrans,
                                detector=d5, wavelength=wavelength)
print("Empty refinement object:", gonioref)

# Let's populate the goniometer refinement object with all control point files:

for fn in npt_files[:]:
    base = os.path.splitext(fn)[0]
    fimg = fabio.open(base + ".edf.gz")
    sg =gonioref.new_geometry(base, image=fimg.data, metadata=fimg.header, control_points=fn, calibrant=LaB6)
    print(base, "Angle:", sg.get_position())
    

print("Filled refinement object:")
print(gonioref)

In [ ]:
# Initial refinement of the goniometer model with 5 dof

gonioref.refine2()

In [ ]:
width=3
height=int(ceil(len(gonioref.single_geometries)/width))
fig,ax = subplots(height, width,figsize=(10,15))
for idx, sg in enumerate(gonioref.single_geometries.values()):
    sg.geometry_refinement.set_param(gonioref.get_ai(sg.get_position()).param)
    jupyter.display(sg=sg, ax=ax[idx//width, idx%width])

In [ ]:
# Final pass of refinement with all constrains removed, very fine refinement

gonioref.bounds = None
gonioref.refine2("slsqp", eps=1e-13, maxiter=10000, ftol=1e-12)

In [ ]:
# Create a MultiGeometry integrator from the refined geometry:

angles = []
images = []
for sg in gonioref.single_geometries.values():
    angles.append(sg.get_position())
    images.append(sg.image)
    
multigeo = gonioref.get_mg(angles)
multigeo.radial_range=(0, 80)
print(multigeo)

In [ ]:
# Integrate the whole set of images in a single run:

res = multigeo.integrate1d(images, 10000)
fig, ax = subplots()
ax.plot(*res)
ax.set_xlabel(res.unit.label)
ax.set_ylabel("Intensity")

#Note the large number of peaks due to hot pixels ....

In [ ]:
# Add hot pixels to the mask: pixel which are 15x more intense than the median in their ring.

thres = 15

old_mask = d5.mask.astype("bool", copy=True)
new_mask = d5.mask.astype("bool", copy=True)

for ai,img in zip(multigeo.ais,images):
    b,a = ai.separate(img, 1000, restore_mask=0)
    b[old_mask] = 0
    b[b<0] = 0
    print(sum(b>thres*a))
    new_mask = numpy.logical_or(new_mask, (b>thres*a))

print(sum(old_mask), sum(new_mask), sum(new_mask)-sum(old_mask))    

In [ ]:
# Update the mask
for ai in multigeo.ais:
    ai.detector.mask = new_mask
    
# Integrate the whole set of images in a single run:
res2 = multigeo.integrate1d(images, 10000)
fig, ax = subplots()
ax.plot(*res, label="Before hot-pixel removal")
ax.plot(*res2, label="After hot-pixel removal")
ax.legend()
ax.set_xlabel(res.unit.label)
ax.set_ylabel("Intensity")

In [ ]:
# Integrate the whole set of images in 2D:

res2d = multigeo.integrate2d(images, 1000, 360)
fig, ax = subplots()
ax.imshow(numpy.arcsinh(res2d[0]), cmap="inferno", origin="lower", aspect="auto",
          extent=[res2d[1][0], res2d[1][-1], res2d[2][0], res2d[2][-1]])
ax.set_xlabel(res.unit.label)
ax.set_ylabel(r"$\chi$ angle")

In [ ]:
print("Total execution time", time.time()-start_time)